In [1]:
import pandas as pd
import numpy as np

import json
import gzip
import zipfile  

In [16]:
with zipfile.ZipFile("/dlabdata1/lugeon/AlexaRankByCategories.json.zip", "r") as z:
    for filename in z.namelist():  
        with z.open(filename) as f:  
            data = f.read()  
            jsondata = json.loads(data)  

In [17]:
root = jsondata['topsites']['category']['Top']

In [18]:
categories = list(root.keys())

In [19]:
categories

['Adult',
 'Arts',
 'Business',
 'Computers',
 'Games',
 'Health',
 'Home',
 'Kids_and_Teens',
 'News',
 'Recreation',
 'Reference',
 'Regional',
 'Science',
 'Shopping',
 'Society',
 'Sports',
 'World',
 '_Most_popular_',
 'Ar',
 'Gam',
 'R']

## Flatten everything

In [6]:
df = pd.DataFrame()

In [7]:
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [8]:
root_f = flatten(root)

<ipython-input-7-72a859df90ce>:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if isinstance(v, collections.MutableMapping):


In [9]:
url = []
cat = []
for k, l in root_f.items():
    url += l
    cat += [k] * len(l)
        

In [10]:
df['url'] = pd.Series(url)
df['cat0'] = pd.Series(cat)

In [11]:
df.drop_duplicates(subset='url', inplace=True)
df.reset_index(inplace=True, drop=True)

In [12]:
df['main_cat'] = df.apply(lambda row: row.cat0.split('_')[0], axis=1) 

In [13]:
df.drop('cat0', inplace=True, axis=1)

In [14]:
df['url'] = df.url.apply(lambda u : 'http://' + u) 

In [15]:
df.head()

,url,main_cat
0,http://cutepet.org,Adult
1,http://jasonmeador.com,Adult
2,http://taremeparadise.com,Adult
3,http://jlist.com,Adult
4,http://jastusa.com,Adult


In [16]:
df.to_csv('/dlabdata1/lugeon/websites_alexa.gz', compression='gzip')

## Most popular only

In [20]:
url = []
cat = []
subcat = []

for c in categories[:16]:
    print(c)
    for k in root[c].keys():
        if k == '_Most_popular_':
            continue
        print("  ", k)
        if type(root[c][k]) == list:
            l = root[c][k]
        else:
            l = root[c][k]['_Most_popular_']
        url += l
        cat += [c] * len(l)
        subcat += [k] * len(l)

Adult
   Arts
   Business
   Computers
   Death_and_Gore
   Games
   Image_Galleries
   News
   Recreation
   Regional
   Shopping
   Society
Arts
   Animation
   Architecture
   Art_History
   Awards
   Bodyart
   Chats_and_Forums
   Classical_Studies
   Comics
   Contests
   Costumes
   Crafts
   Cultures_and_Groups
   Design
   Digital
   Directories
   Education
   Entertainment
   Events
   Genres
   Graphic_Design
   Humanities
   Illustration
   Literature
   Magazines_and_E-zines
   Movies
   Music
   News_and_Media
   Online_Writing
   Organizations
   People
   Performing_Arts
   Periods_and_Movements
   Personal_Pages
   Photography
   Radio
   Regional
   Television
   Video
   Visual_Arts
   Web_Rings
   Weblogs
   Writers_Resources
Business
   Accounting
   Aerospace_and_Defense
   Agriculture_and_Forestry
   Arts_and_Entertainment
   Associations
   Automotive
   Biotechnology_and_Pharmaceuticals
   Business_and_Society
   Business_Services
   Chemicals
   Construction_a

In [21]:
df = pd.DataFrame({'url': url, 'cat0': cat, 'subcat0': subcat})

In [22]:
df['url'] = df.url.apply(lambda u : 'http://www.' + u) 

In [23]:
df

,url,cat0,subcat0
0,http://www.nhentai.net,Adult,Arts
1,http://www.furaffinity.net,Adult,Arts
2,http://www.literotica.com,Adult,Arts
3,http://www.iafd.com,Adult,Arts
4,http://www.sofurry.com,Adult,Arts
...,...,...,...
20056,http://www.nays.org,Sports,Youth_and_High_School
20057,http://www.nhsca.com,Sports,Youth_and_High_School
20058,http://www.naysi.com,Sports,Youth_and_High_School
20059,http://www.chap.com,Sports,Youth_and_High_School


In [37]:
df.to_csv('/dlabdata1/lugeon/websites_alexa_mostpop2.gz', compression='gzip')

In [2]:
df = pd.read_csv('/dlabdata1/lugeon/websites_alexa_mostpop2.gz', index_col=0)

In [3]:
df.head()

,url,cat0,subcat0
0,http://www.nhentai.net,Adult,Arts
1,http://www.furaffinity.net,Adult,Arts
2,http://www.literotica.com,Adult,Arts
3,http://www.iafd.com,Adult,Arts
4,http://www.sofurry.com,Adult,Arts


In [32]:
count = df.cat0.value_counts()
count

Sports            2969
Business          2221
Computers         1863
Shopping          1664
Arts              1531
Society           1480
Recreation        1445
Health            1407
Science           1130
Games              805
Reference          712
News               703
Home               596
Kids_and_Teens     567
Regional           500
Adult              468
Name: cat0, dtype: int64

In [33]:
biggest_cat = count[count >= 1000].index.values
biggest_cat

array(['Sports', 'Business', 'Computers', 'Shopping', 'Arts', 'Society',
       'Recreation', 'Health', 'Science'], dtype=object)

In [34]:
df_red = df[df.apply(lambda row: row.cat0 in biggest_cat, axis=1)]

In [35]:
df_red

,url,cat0,subcat0
468,http://www.sankakucomplex.com,Arts,Animation
469,http://www.anime-planet.com,Arts,Animation
470,http://www.pokemon.com,Arts,Animation
471,http://www.viz.com,Arts,Animation
472,http://www.animefreak.tv,Arts,Animation
...,...,...,...
20056,http://www.nays.org,Sports,Youth_and_High_School
20057,http://www.nhsca.com,Sports,Youth_and_High_School
20058,http://www.naysi.com,Sports,Youth_and_High_School
20059,http://www.chap.com,Sports,Youth_and_High_School


In [44]:
categories = df_red.cat0.unique()

tot_samples = 1500
frac = 0.2
sep = int(tot_samples * frac)

df_train = pd.DataFrame()
df_test = pd.DataFrame()

for cat in df_red.cat0.unique():
    df_cat = df_red[df_red.cat0 == cat]
    if df_cat.shape[0] < tot_samples:
        replace=True
    else:
        replace=False
    df_cat = df_cat.sample(tot_samples, replace=replace)
    ix = np.arange(tot_samples)
    np.random.shuffle(ix)
    
    df_train = pd.concat((df_train, df_cat.iloc[ix[sep:]]))
    df_test = pd.concat((df_test, df_cat.iloc[ix[:sep]]))
    
    
    

In [47]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [48]:
df_train

,url,cat0,subcat0
0,http://www.tccalc.narod.ru,Arts,Video
1,http://www.iaac.net,Arts,Architecture
2,http://www.chicagomanualofstyle.org,Arts,Writers_Resources
3,http://www.chanel.com,Arts,Design
4,http://www.nyfa.org,Arts,Organizations
...,...,...,...
10795,http://www.crowsdarts.com,Sports,Darts
10796,http://www.balealsurfcamp.com,Sports,Water_Sports
10797,http://www.biathlon-ruhpolding.de,Sports,Multi-Sports
10798,http://www.volleyball.com,Sports,Volleyball


In [49]:
df_test

,url,cat0,subcat0
0,http://www.nationaltheatre.org.uk,Arts,Performing_Arts
1,http://www.artsjournal.com,Arts,Magazines_and_E-zines
2,http://www.riksha.com,Arts,Magazines_and_E-zines
3,http://www.arthouseproductions.org,Arts,Organizations
4,http://www.awards.the-aop.org,Arts,Awards
...,...,...,...
2695,http://www.triathloncanada.com,Sports,Multi-Sports
2696,http://www.wakeboardingmag.com,Sports,Water_Sports
2697,http://www.spohnranch.com,Sports,Skateboarding
2698,http://www.uskorfball.org,Sports,Korfball


In [50]:
df_train.to_csv('/dlabdata1/lugeon/websites_alexa_mp2_1500_9cat_train.gz', compression='gzip')
df_test.to_csv('/dlabdata1/lugeon/websites_alexa_mp2_1500_9cat_test.gz', compression='gzip')